In [1]:
import cudf
import cupy
import pandas as pd
import numpy as np

In [2]:
from lib.data import load_dataset

x_train, y_train, x_test, y_test = load_dataset('dts/train.csv')

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
rs = 2
forest = Pipeline([
    ('feature_selection', SelectFromModel(RandomForestClassifier( random_state=rs, n_estimators=10).fit(x_train, y_train))),
    ('classification', RandomForestClassifier(random_state=rs, n_estimators=10))
])

In [4]:
forest.fit(x_train, y_train)
y_pred = forest.predict(x_test)

In [5]:
from lib.metrics import metrics
metrics(y_pred, y_test)


Metrics: 
 Acc: 0.95, ROC: 0.95
 PRE: 0.97, REC: 0.92


In [6]:
df2 = pd.read_csv("dts/test.csv")

x_test = df2.iloc[:, :-1]
y_pred = forest.predict(x_test)

In [7]:
x_test.shape

(2000, 784)

In [8]:
from lib.data import export_results
result = pd.DataFrame(data={
        "Id": range(y_pred.shape[0]),
        "Category": y_pred.astype(int)
    }, index=None)

result.to_csv("result.csv", index=None)
# export_results(y_pred)

In [9]:
# import time
# from sklearn.inspection import permutation_importance
# from sklearn.tree import DecisionTreeClassifier

# clf = DecisionTreeClassifier(random_state=rs)
# clf.fit(x_train, y_train)

# importance = clf.feature_importances_

# # for i,v in enumerate(importance):
# # 	print('Feature: %0d, Score: %.5f' % (i,v))

In [10]:
# from sklearn.feature_selection import SelectFromModel

# model = SelectFromModel(DecisionTreeClassifier( random_state=rs).fit(x_train, y_train), prefit=True)
# x_new = model.transform(x_train)
# clf.fit(x_new, y_train)
# x_new_test = model.transform(x_test)
# y_pred = clf.predict(x_new_test)
